# system for running multiple players at once

## imports


In [ ]:
import os
import mido
import random
import pretty_midi
from mido import MidiFile, Message
import threading
from threading import Thread, Event
from queue import Queue
import simpleaudio as sa
from pathlib import Path
from datetime import datetime
import time
import matplotlib.pyplot as plt
import numpy as np
import wave

## player class


In [ ]:
class SimplePlayer:
    do_tick = True

    def __init__(
        self,
        name: str,
        sound: str,
        kill_event: Event,
        playback_event: Event,
        play_event: Event,
        filename_queue: Queue,
        outport: str = "Disklavier",
    ) -> None:
        self.name = name
        self.sound = sound
        self.play = play_event
        self.kill_event = kill_event
        self.get_next = playback_event
        self.file_queue = filename_queue
        self.outport = outport

    def playback_loop(self):
        self.get_next.set()

        while not self.kill_event.is_set():
            # get next file from queue
            print(
                f"[{datetime.now().strftime('%H:%M:%S.%f')}] {self.name} waiting for file"
            )
            self.playing_file_path = self.file_queue.get()
            self.playing_file = os.path.basename(self.playing_file_path)
            self.get_next.set()

            # print progress
            file_tempo = int(os.path.basename(self.playing_file).split("-")[1])
            found_tempo = -1
            for track in MidiFile(self.playing_file_path).tracks:
                for msg in track:
                    if msg.type == "set_tempo":
                        found_tempo = msg.tempo
            print(
                f"[{datetime.now().strftime('%H:%M:%S.%f')}] {self.name} playing '{self.playing_file}' ({file_tempo}BPM --> {round(mido.tempo2bpm(found_tempo)):01d}BPM)"
            )

            # play file
            self.play_midi_file(self.playing_file_path)

        print(f"[{datetime.now().strftime('%H:%M:%S.%f')}] {self.name} shutting down")

    def play_midi_file(self, midi_path: str) -> None:
        midi = MidiFile(midi_path)

        with mido.open_output(self.outport) as outport:  # type: ignore
            print(
                f"[{datetime.now().strftime('%H:%M:%S.%f')}] {self.name} waiting to play"
            )
            while not self.play.is_set():
                time.sleep(0.0000001)

            self.play.clear()
            print(f"[{datetime.now().strftime('%H:%M:%S.%f')}] {self.name} playing")

            last_beat = time.time()
            for msg in midi.play(meta_messages=True):
                if not msg.is_meta:
                    outport.send(msg)
                else:
                    if msg.type == "set_tempo":  # type: ignore
                        print(f"\t\tplaying at {mido.tempo2bpm(msg.tempo)} BPM", msg)  # type: ignore
                    if msg.type == "text":  # type: ignore
                        beat = time.time()
                        print(
                            f"[{datetime.now().strftime('%H:%M:%S.%f')}] {self.name} {msg.text} ({beat - last_beat:.05f}s)"
                        )
                        last_beat = beat
                        sa.WaveObject.from_wave_file(self.sound).play()

                # end active notes and return if killed
                if self.kill_event.is_set():
                    with mido.open_output(self.outport) as outport:  # type: ignore
                        for note in range(128):
                            msg = Message("note_off", note=note, velocity=0, channel=0)
                            outport.send(msg)
                    break

## helper functions


In [ ]:
def change_tempo(midi_file_path: str, new_bpm: int = 80) -> str:
    midi = mido.MidiFile(midi_file_path)
    new_tempo = mido.bpm2tempo(new_bpm)
    new_message = mido.MetaMessage("set_tempo", tempo=new_tempo, time=0)
    tempo_added = False

    for track in midi.tracks:
        for msg in track:
            if msg.type == "set_tempo":
                track.remove(msg)
                # print(f"removed set tempo message", msg)

        if not tempo_added:
            track.insert(0, new_message)
            # print(f"adding message (tempo={new_bpm}) {new_message}")
            tempo_added = True

    # if no tracks had a set_tempo message and no new one was added, add a new track with the tempo message
    if not tempo_added:
        new_track = mido.MidiTrack()
        print(f"adding message to new track {new_message}")
        new_track.append(new_message)
        midi.tracks.append(new_track)

    new_file_path = os.path.join("tmp/playlist", f"{Path(midi_file_path).stem}.mid")
    midi.save(new_file_path)

    return new_file_path

In [ ]:
def draw_piano_roll(piano_roll, beats=[0], fs=100, title="Piano Roll") -> None:
    plt.style.use("dark_background")
    plt.figure(figsize=(12, 8))
    plt.imshow(
        piano_roll, aspect="auto", origin="lower", cmap="magma", interpolation="nearest"
    )
    plt.vlines([b * fs for b in beats], 0, 120, "g", linewidth=1, alpha=0.4)
    plt.title(title)
    plt.xlabel("Time (seconds)")
    plt.ylabel("MIDI Note Number")
    plt.colorbar()

    tick_spacing = 1
    ticks = np.arange(0, len(piano_roll.T) / fs, tick_spacing)
    plt.xticks(ticks * fs, labels=[f"{int(tick)}" for tick in ticks])
    plt.show()

In [ ]:
def calc_beats(tempo: int, start_time_seconds: float, end_time_seconds: float):
    beat_duration = 60 / tempo
    current_time = start_time_seconds
    beat_times = []

    while current_time <= end_time_seconds:
        beat_times.append(current_time)
        current_time += beat_duration

    return beat_times

In [ ]:
def metronome(
    tempo: int, ready_event: Event, go_event: Event, kill_event: Event, bps: int = 8
):
    tick_rate = 60.0 / tempo

    print(f"ticking every {tick_rate:.01f} seconds")

    beats = 0
    start_time = time.time()
    last_beat = start_time
    while not kill_event.is_set():
        beat = time.time()
        if beat - last_beat >= tick_rate:
            if beats // bps:
                print(
                    f"[{datetime.now().strftime('%H:%M:%S.%f')}] beat {beats}    ({beat - last_beat:.05f}s)\tgo!"
                )
                go_event.set()
                beats = 0
            elif beats // (bps - 1):
                print(
                    f"[{datetime.now().strftime('%H:%M:%S.%f')}] beat {beats}    ({beat - last_beat:.05f}s)\tready?"
                )
                ready_event.set()
            else:
                print(
                    f"[{datetime.now().strftime('%H:%M:%S.%f')}] beat {beats}    ({beat - last_beat:.05f}s)"
                )

            sa.WaveObject.from_wave_file("../data/m_kick.wav").play()
            last_beat = beat
            beats += 1
    print("metronome shutting down")

## build players


### setup players


In [ ]:
kill_p1 = Event()
give_p1 = Event()
plst_p1 = Queue()
kill_p2 = Event()
give_p2 = Event()
plst_p2 = Queue()
play_event = Event()

player1 = SimplePlayer(
    "p1", "../data/m_tick.wav", kill_p1, give_p1, play_event, plst_p1, "to Max 1"
)
thread_p1 = Thread(target=player1.playback_loop, args=(), name="p1")

player2 = SimplePlayer(
    "p2", "../data/m_hat.wav", kill_p2, give_p2, play_event, plst_p2, "to Max 1"
)
thread_p2 = Thread(target=player2.playback_loop, args=(), name="p2")

### setup control metronome


In [ ]:
kill_metro = Event()
ready_event = Event()

bpm = 60

thread_metro = Thread(target=metronome, args=(bpm, ready_event, play_event, kill_metro))

### play


In [ ]:
for file in os.listdir("tmp/playlist"):
    os.remove(os.path.join("tmp/playlist", file))

dataset_folder = "../data/datasets/careful"
all_segments = os.listdir(dataset_folder)
p1_main = True

# init threads
thread_p1.start()
thread_p2.start()
thread_metro.start()

# play
num_tracks = 0
while num_tracks < 5:
    if ready_event.is_set():
        # ready next file
        segment = all_segments[random.randint(0, len(all_segments))]
        seg_path = os.path.join(dataset_folder, segment)
        t_file = change_tempo(seg_path, bpm)

        # send to either p1 or p2
        if p1_main:
            print(
                f"[{datetime.now().strftime('%H:%M:%S.%f')}] putting next file in p1's queue: {t_file}",
                give_p1.is_set(),
            )
            plst_p1.put(t_file)
            give_p1.clear()
        else:
            print(
                f"[{datetime.now().strftime('%H:%M:%S.%f')}] putting next file in p2's queue: {t_file}",
                give_p2.is_set(),
            )
            plst_p2.put(t_file)
            give_p2.clear()

        p1_main = not p1_main
        ready_event.clear()
        num_tracks += 1

        # plot
        # pm = pretty_midi.PrettyMIDI(seg_path)
        # tempo = int(segment.split("-")[1])
        # beats = calc_beats(tempo, 0.0, pm.get_end_time())
        # draw_piano_roll(
        #     pm.get_piano_roll(),
        #     beats=[b - beats[0] for b in beats],
        #     title=segment,
        # )
        pmt = pretty_midi.PrettyMIDI(t_file)
        beatst = calc_beats(bpm, 0.0, pmt.get_end_time())
        draw_piano_roll(
            pmt.get_piano_roll(),
            beats=[b - beatst[0] for b in beatst],
            title=f"shifted to {bpm}BPM",
        )

# kill threads
kill_p1.set()
thread_p1.join()
kill_p2.set()
thread_p2.join()
kill_metro.set()
thread_metro.join()